<a href="https://colab.research.google.com/github/mikasa42/desafio-junior-data-scientist/blob/main/Analise_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Desafio Júnior

## Instalando depedências

In [2]:
!pip install pandas
!pip install pandas-gbq
!pip install google-cloud-bigquery
!pip install pyarrow
!pip install basedosdados -q
import basedosdados as bd
import pandas as pd
import plotly.express as px

# Billing project ID (Google Cloud Platform)
PROJECT_ID = "refined-legend-454101-p5"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.5 MB/s eta 0:00:00


## Queries para trazer todos os dados utilizados na base de dados do DataRio

In [144]:
query_chamados_01_04_2023 = "SELECT * FROM `datario.adm_central_atendimento_1746.chamado` WHERE DATE(data_inicio) = '2023-04-01'"
query_bairros = "SELECT * FROM `datario.dados_mestres.bairro`"
query_all_chamados_2022_2023 = "SELECT data_inicio, tipo, subtipo FROM `datario.adm_central_atendimento_1746.chamado` WHERE DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'"
query_chamados_sossego_2022_2023 = "SELECT * FROM `datario.adm_central_atendimento_1746.chamado` WHERE tipo = 'Perturbação do sossego' AND data_inicio BETWEEN '2022-01-01' AND '2023-12-31'"
query_eventos = "SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`"
query_chamados_sossego_eventos = """
SELECT
    c.id_chamado,
    c.data_inicio AS data_chamado,
    e.evento
FROM `datario.adm_central_atendimento_1746.chamado` AS c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` AS e
    ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.tipo = "Perturbação do sossego"
AND e.data_inicial IS NOT NULL
AND e.data_final IS NOT NULL
AND c.data_inicio BETWEEN '2022-01-01 00:00:00' AND '2023-12-31 00:00:00'
ORDER BY c.data_inicio ASC;
"""
query_chamados_sossego_eventos_media = """
SELECT
    e.evento,
    EXTRACT(YEAR FROM e.data_inicial) AS ano_evento,
    COUNT(c.id_chamado) AS total_chamados,
    DATE_DIFF(e.data_final, e.data_inicial, DAY) AS duracao_evento_dias,
    COUNT(c.id_chamado) / (DATE_DIFF(e.data_final, e.data_inicial, DAY) + 1) AS media_diaria_chamados
FROM `datario.adm_central_atendimento_1746.chamado` AS c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` AS e
    ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.tipo = "Perturbação do sossego"
AND e.data_inicial IS NOT NULL
AND e.data_final IS NOT NULL
AND c.data_inicio BETWEEN '2022-02-01' AND '2023-12-31'
GROUP BY e.evento, e.data_inicial, e.data_final
ORDER BY media_diaria_chamados DESC;
"""
query_chamados_sossego_eventos_media_diaria = """
WITH eventos_chamados AS (
    SELECT
        e.evento,
        COUNT(c.id_chamado) AS total_chamados,
        DATE_DIFF(e.data_final, e.data_inicial, DAY) + 1 AS duracao_evento_dias
    FROM `datario.adm_central_atendimento_1746.chamado` AS c
    JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` AS e
        ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
    WHERE c.tipo = "Perturbação do sossego"
        AND e.data_inicial IS NOT NULL
        AND e.data_final IS NOT NULL
        AND c.data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
    GROUP BY e.evento, e.data_inicial, e.data_final
),

media_geral AS (
    SELECT COUNT(*) / 730 AS media_diaria_geral
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE tipo = "Perturbação do sossego"
    AND data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
)

SELECT
    e.evento,
    SUM(e.total_chamados) AS total_chamados_evento,
    SUM(e.duracao_evento_dias) AS total_dias_evento,
    SUM(e.total_chamados) / SUM(e.duracao_evento_dias) AS media_diaria_evento,
    mg.media_diaria_geral,
    (SUM(e.total_chamados) / SUM(e.duracao_evento_dias)) / mg.media_diaria_geral AS fator_variacao,
    ((SUM(e.total_chamados) / SUM(e.duracao_evento_dias)) / mg.media_diaria_geral - 1) * 100 AS variacao_percentual
FROM eventos_chamados e
CROSS JOIN media_geral mg
GROUP BY e.evento, mg.media_diaria_geral
ORDER BY variacao_percentual DESC;
"""

## Carregando os Dataframes

In [145]:
# Executa queries e cria DataFrames pandas de todos os resultados
PROJECT_ID = 'refined-legend-454101-p5'
# Chamados realizados no dia 01/04/2023
bd_df_chamados_01_04_2023 = bd.read_sql(query_chamados_01_04_2023, billing_project_id=PROJECT_ID)
df_chamados_01_04_2023= pd.DataFrame(data = bd_df_chamados_01_04_2023)

# Tabela de bairros
bd_df_bairros = bd.read_sql(query_bairros, billing_project_id=PROJECT_ID)
df_bairros = pd.DataFrame(data = bd_df_bairros)

# Todos os chamados entre 2022 e 2023
bd_df_chamados_2022_2023 = bd.read_sql(query_all_chamados_2022_2023, billing_project_id=PROJECT_ID)
df_chamados_2022_2023 = pd.DataFrame(data = bd_df_chamados_2022_2023)

#Todos os chamados com o tipo "Pertubação do sossego"
bd_df_chamados_sossego_2022_2023 = bd.read_sql(query_chamados_sossego_2022_2023, billing_project_id=PROJECT_ID)

# Tabela de eventos
bd_df_eventos = bd.read_sql(query_eventos, billing_project_id=PROJECT_ID)
df_eventos = pd.DataFrame(data = bd_df_eventos)

# Todos os chamados com o tipo "pertubação de sossego" em eventos
bd_df_chamados_sossego_eventos = bd.read_sql(query_chamados_sossego_eventos, billing_project_id=PROJECT_ID)
df_chamados_sossego_eventos = pd.DataFrame(data = bd_df_chamados_sossego_eventos)

# Maior média em eventos
bd_df_chamados_sossego_eventos_media = bd.read_sql(query_chamados_sossego_eventos_media, billing_project_id=PROJECT_ID)
bd_df_chamados_sossego_eventos_media = pd.DataFrame(data = bd_df_chamados_sossego_eventos_media)

#Medias diarias de chamados em eventos comparando a média de chamados no período 01/01/2022 até 31/12/2023
bd_df_chamados_sossego_eventos_media_diarias = bd.read_sql(query_chamados_sossego_eventos_media_diaria, billing_project_id=PROJECT_ID)
bd_df_chamados_sossego_eventos_media_diarias = pd.DataFrame(data = bd_df_chamados_sossego_eventos_media_diarias)


Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|


Função com o gráfico utilizado

In [146]:

import plotly.express as px

def plot_bar(data, x, y, color, text, title, subtitle, labels, color_discrete_sequence=None, orientation='v'):
  fig = px.bar(data,
               x = x,
               y = y,
               color = color,
               text = text,
               title=title + f' {subtitle}',
               labels=labels,
               color_discrete_sequence=color_discrete_sequence)
  fig.update_traces(textposition='outside', showlegend=False)
  fig.update_layout(width=1080, bargap=0.2)
  fig.show()

## Questão 1)
Quantos chamados foram abertos no dia 01/04/2023?

In [147]:


print("Foram abertos %s chamados no dia 01/04/2023" %(len(df_chamados_01_04_2023.values)))



Foram abertos 1903 chamados no dia 01/04/2023


Histograma para comparar o dia de 01/04/2023 com o mês inteiro

In [148]:
chamados_04_2023 = df_chamados_2022_2023.loc[df_chamados_2022_2023['data_inicio'].between('2023-04-01', '2023-04-30')]
chamados_04_2023['data_inicio'] = chamados_04_2023['data_inicio'].dt.date

chamados_04_2023_por_dia = chamados_04_2023.groupby('data_inicio')['data_inicio'].count().reset_index(name='chamados')


plot_bar(data=chamados_04_2023_por_dia,
         x='data_inicio',
         y='chamados',
         color='data_inicio',
         text='chamados',
         title='Chamados por dia nos dias 01/04/23 a 30/04/23',
         subtitle='1903 chamados foram abertos no dia 01/04/23',
         labels={'data_inicio': 'Dia', 'chamados': 'Chamados'},
         #color_discrete_sequence=[BLUE_HIGHLIGHT, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY, LIGHT_GRAY]
         )



##Questão 2)
Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [149]:
# Agrupa por tipo
group_by_tipo = df_chamados_01_04_2023.groupby('tipo')

# Contabiliza os chamados por tipo
#count_tipo = group_by_tipo['tipo'].count()


# Agrupa por tipo e conta os chamados
count_tipo = df_chamados_01_04_2023.groupby('tipo').size().reset_index(name='chamados')

# Plota o gráfico de barras com os dados agregados
fig = px.histogram(count_tipo,
                   x='chamados',
                   y='tipo',
                   orientation='h',
                   title='Distribuição de Chamados por Tipo')
fig.update_layout(bargap=0.2)
fig.show()

count_tipo = group_by_tipo['tipo'].count()

# Nome do tipo com mais chamados na variavel tipo para exibição
tipo = count_tipo[count_tipo == count_tipo.max()].index[0]

# Maior valor da série de quantidade de chamados por tipo
max_value = count_tipo.max()
print("\n")
print("O tipo de chamado com mais ocorrências no dia 01/04/2023 foi o '%s' com %d chamados abertos" % (tipo, count_tipo.max()))
print("\n")




O tipo de chamado com mais ocorrências no dia 01/04/2023 foi o 'Estacionamento irregular' com 373 chamados abertos




##Questão 3)
Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [150]:
# Merge, similar a inner join do SQL, dos DataFrames de Chamados e Bairros
df_merge_chamados_bairros = pd.merge(df_chamados_01_04_2023, df_bairros, on='id_bairro')

# Agrupa os dados por bairro e quantidade de chamados de cada bairro
df_count_chamados_bairros = df_merge_chamados_bairros.groupby('nome').count()

# Seleciona a série de valores de bairro e quantidade de chamados abertos em ordem decrescente
chamados_bairros_decrescente = df_count_chamados_bairros['id_chamado'].sort_values(ascending=False)

# Seleciona os 10 bairros com maior número de chamados
top_10_bairros = chamados_bairros_decrescente.iloc[0:10]

# Transforma os dados em um DataFrame
df_top_10_bairros = top_10_bairros.reset_index()

# Plota o gráfico de barras com os dados dos top 10 bairros
fig = px.histogram(df_top_10_bairros,
                   x='nome',
                   y='id_chamado',
                   title='Top 10 bairros com mais chamados')
fig.update_layout(bargap=0.2)
fig.show()

# Armazena os 3 bairros com mais chamados abertos no dia 01/04/2023
resposta_3 = chamados_bairros_decrescente.iloc[0:3]

print("Os 3 bairros com mais chamados abertos neste dia foram:\n")
print(resposta_3)

Os 3 bairros com mais chamados abertos neste dia foram:

nome
Campo Grande       124
Tijuca              96
Barra da Tijuca     60
Name: id_chamado, dtype: int64


##Questão 4)
Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [151]:
# Agrupa os dados por bairro e quantidade de chamados de cada bairro
df_count_chamados_subprefeitura = df_merge_chamados_bairros.groupby('subprefeitura').count()

# Seleciona a série de valores de bairro e quantidade de chamados abertos em ordem decrescente
chamados_subprefeitura_decrescente = df_count_chamados_subprefeitura['id_chamado'].sort_values(ascending=False)

# Armazena a subprefeitura com mais chamados abertos no dia 01/04/2023
resposta_4 = chamados_subprefeitura_decrescente.iloc[0:1]


print("A subprefeitura com mais chamados abertos neste dia foi:\n")
print(resposta_4)



A subprefeitura com mais chamados abertos neste dia foi:

subprefeitura
Zona Norte    534
Name: id_chamado, dtype: int64


##Questão 5)
Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

 Sim, existe 131 chamados da tabela Chamados aberto nesse dia que não possui id_bairro (valor null na coluna id_bairro) e, por isso, não pode ser
associado a nenhum registro da tabela Bairro, o que inclui tanto bairro quanto subprefeitura.

In [152]:

df_merge_left_chamados_bairros = pd.merge(df_chamados_01_04_2023, df_bairros, how='left',  on='id_bairro')


# Seleciona chamados com bairro = null, ou seja, chamados não associados a um bairro
df_merge_chamados_bairros_null = df_merge_left_chamados_bairros[df_merge_left_chamados_bairros['nome'].isnull()]
contagem_id_bairro_null = df_merge_chamados_bairros_null.shape[0]

# Verifica se há registros no DataFrame
if not df_merge_chamados_bairros_null.empty:
    print(f"Número de registros com id_bairro = null: {contagem_id_bairro_null}")
else:
    print("Não há chamados com 'id_bairro' nulo no DataFrame.")



Número de registros com id_bairro = null: 131


##Questão 6)
Quantos chamados de Perturbação do sossego foram abertos nesse período?

In [153]:
df_chamados_sossego_2022_2023 = pd.DataFrame(data = bd_df_chamados_sossego_2022_2023)
# Como se trata de um conjunto de dados muito grande o filto foi realizado diretamente na query SQL.
print("O número de chamados com tipo 'Perturbação do sossego' abertos desde 01/01/2022 até 31/12/2023 é:", len(df_chamados_sossego_2022_2023))



O número de chamados com tipo 'Perturbação do sossego' abertos desde 01/01/2022 até 31/12/2023 é: 66051


##Questão 7)
Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [154]:
print("Os chamados de subtipo 'Perturbação do sossego' abertos durante os eventos contidos na tabela de eventos foram:\n")
df_chamados_sossego_eventos[['id_chamado','data_chamado', 'evento']]

Os chamados de subtipo 'Perturbação do sossego' abertos durante os eventos contidos na tabela de eventos foram:



,id_chamado,data_chamado,evento
0,17658238,2022-09-02 00:00:50,Rock in Rio
1,17658242,2022-09-02 00:11:44,Rock in Rio
2,17658243,2022-09-02 00:12:20,Rock in Rio
3,17658249,2022-09-02 00:26:20,Rock in Rio
4,17658250,2022-09-02 00:26:41,Rock in Rio
...,...,...,...
984,19659581,2023-12-30 21:10:08,Réveillon
985,19659589,2023-12-30 21:27:58,Réveillon
986,19659645,2023-12-30 22:30:44,Réveillon
987,19659656,2023-12-30 22:49:26,Réveillon


##Questão 8)
Quantos chamados desse subtipo foram abertos em cada evento?

In [155]:
chamados_tipos = df_chamados_sossego_eventos.groupby(['evento']).count()

print("O número de chamados desse subtipo abertos em cada evento foram: \n")
chamados_tipos['id_chamado']


O número de chamados desse subtipo abertos em cada evento foram: 



,id_chamado
evento,
Carnaval,221
Rock in Rio,633
Réveillon,135


##Questão 9)
Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [156]:
bd_df_chamados_sossego_eventos_media[['evento','ano_evento','total_chamados','duracao_evento_dias','media_diaria_chamados']]
print("\n")
print("O evento com a maior média diária de chamados abertos desse subtipo foi:\n")
bd_df_chamados_sossego_eventos_media.head(1)



O evento com a maior média diária de chamados abertos desse subtipo foi:



,evento,ano_evento,total_chamados,duracao_evento_dias,media_diaria_chamados
0,Rock in Rio,2022,386,3,96.5


##Questão 10)
Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2024.

In [157]:
bd_df_chamados_sossego_eventos_media_diarias[['evento','total_chamados_evento','total_dias_evento','media_diaria_evento','media_diaria_geral','fator_variacao','variacao_percentual']]



,evento,total_chamados_evento,total_dias_evento,media_diaria_evento,media_diaria_geral,fator_variacao,variacao_percentual
0,Rock in Rio,633,7,90.428571,90.480822,0.999423,-0.057748
1,Carnaval,221,4,55.250000,90.480822,0.610627,-38.937336
2,Réveillon,135,6,22.500000,90.480822,0.248671,-75.132852
